In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px

In [5]:
data_path = '../../../Data/KU-HAR_time_domain_subsamples_20750x300.csv'
df = pd.read_csv(data_path)
df.head()

,0.0042305,-0.00028038,-0.013906,-0.0025436,0.027433,0.0583,0.05167,0.071857,0.080653,0.047917,...,-0.016692,-0.018566,-0.017361,-0.017919,-0.01668,-0.01665,-0.013097,0,300,1
0,0.012482,0.032496,0.067856,0.071552,0.078103,0.040353,-0.001059,-0.018580,-0.026813,-0.011280,...,-0.008210,-0.010408,-0.011459,-0.011747,-0.010394,-0.008070,-0.004354,0,300,2
1,0.012127,0.028458,0.062075,0.052611,0.022942,-0.010017,-0.023151,-0.028515,0.005036,0.008450,...,0.000915,0.000771,-0.002560,-0.003020,-0.004190,0.000215,0.000850,0,300,3
2,0.052964,0.074319,0.110670,0.108490,0.091825,0.056989,0.029337,-0.016724,-0.042265,-0.053983,...,-0.001209,0.002878,0.000663,0.000982,-0.002148,-0.009200,-0.010653,0,300,4
3,-0.020462,-0.030787,-0.008617,0.008906,0.045046,0.042136,0.045037,0.045182,0.025113,0.021730,...,-0.006369,-0.006614,-0.004698,-0.007279,-0.006861,-0.006161,-0.006113,0,300,5
4,0.039138,0.037687,0.022782,0.009661,-0.000885,-0.000630,-0.019837,-0.016246,-0.007580,0.001479,...,0.001930,0.004500,0.008865,0.011290,0.010965,0.010364,0.009872,0,300,6


In [6]:
subdf_list = []
label_dict = {'ID': [], 'label': []}

for i, r in df.iterrows():
    r = r.values 
    acx, acy, acz, gyx, gyy, gyz, label, _, ID = r[:300], r[300:600], r[600:900], r[900:1200], r[1200:1500], r[1500:1800], r[1800], r[1801], r[1802]
    sub_df = pd.DataFrame({'accelX': acx, 'accelY': acy, 'accelZ': acz, 'GyroX': gyx, 'GyroY': gyy, 'GyroZ': gyz}, index=[int(ID),]*300)
    label_dict['label'].append(int(label))
    label_dict['ID'].append(int(ID))
    subdf_list.append(sub_df)
    label_df = pd.DataFrame(label_dict)
    label_df.set_index('ID')

full_df = pd.concat(subdf_list)
full_df.head()

,accelX,accelY,accelZ,GyroX,GyroY,GyroZ
2,0.012482,-0.081862,0.007547,-0.024319,-0.010539,-0.007933
2,0.032496,-0.087494,0.042496,-0.025171,-0.003985,-0.006765
2,0.067856,-0.054918,0.071386,-0.023936,0.001559,-0.003346
2,0.071552,-0.030374,0.062229,-0.022035,0.011073,-0.003037
2,0.078103,-0.012147,0.066126,-0.021341,0.020339,-0.005582


In [8]:
from feature_engine.outliers import Winsorizer
import plotly.express as px

In [9]:
# set up the capper
capper = Winsorizer(capping_method='gaussian', tail='both', fold=3, variables=list(full_df.columns))

# fit the capper
capper.fit(full_df)

Winsorizer(tail='both',
           variables=['accelX', 'accelY', 'accelZ', 'GyroX', 'GyroY', 'GyroZ'])

In [10]:
# transform the data
train_t= capper.transform(full_df)

In [11]:
train_t.head()

,accelX,accelY,accelZ,GyroX,GyroY,GyroZ
2,0.012482,-0.081862,0.007547,-0.024319,-0.010539,-0.007933
2,0.032496,-0.087494,0.042496,-0.025171,-0.003985,-0.006765
2,0.067856,-0.054918,0.071386,-0.023936,0.001559,-0.003346
2,0.071552,-0.030374,0.062229,-0.022035,0.011073,-0.003037
2,0.078103,-0.012147,0.066126,-0.021341,0.020339,-0.005582


In [12]:
class_dict = dict(zip(label_dict['ID'], label_dict['label']))
train_t['action'] = [class_dict[x] for x in full_df.index]
train_t.head()

,accelX,accelY,accelZ,GyroX,GyroY,GyroZ,action
2,0.012482,-0.081862,0.007547,-0.024319,-0.010539,-0.007933,0
2,0.032496,-0.087494,0.042496,-0.025171,-0.003985,-0.006765,0
2,0.067856,-0.054918,0.071386,-0.023936,0.001559,-0.003346,0
2,0.071552,-0.030374,0.062229,-0.022035,0.011073,-0.003037,0
2,0.078103,-0.012147,0.066126,-0.021341,0.020339,-0.005582,0


In [14]:
train_t.head()

,accelX,accelY,accelZ,GyroX,GyroY,GyroZ,action
2,0.012482,-0.081862,0.007547,-0.024319,-0.010539,-0.007933,0
2,0.032496,-0.087494,0.042496,-0.025171,-0.003985,-0.006765,0
2,0.067856,-0.054918,0.071386,-0.023936,0.001559,-0.003346,0
2,0.071552,-0.030374,0.062229,-0.022035,0.011073,-0.003037,0
2,0.078103,-0.012147,0.066126,-0.021341,0.020339,-0.005582,0


In [18]:
accelx_df = train_t.loc[:, ['accelX', 'action']].reset_index()
accelx_df.head()

,index,accelX,action
0,2,0.012482,0
1,2,0.032496,0
2,2,0.067856,0
3,2,0.071552,0
4,2,0.078103,0


In [23]:
accelx_df = train_t.reset_index().loc[:, ['index' ,'accelX', 'action']].groupby('index').sample(n=1).reset_index(drop=True)
accelx_df.shape

(20749, 3)

In [24]:
accelx_df.head()

,index,accelX,action
0,2,0.026719,0
1,3,0.026985,0
2,4,0.006589,0
3,5,0.024808,0
4,6,0.022782,0


In [ ]:
accelx_df = train_t.loc[:, ['accelX', 'action']].groupby('index').sample(n=1).reset_index()

# df = px.data.tips()
fig = px.histogram(accelx_df, x="accelX", color="action", marginal="rug")
fig.show()